Dictionary by hand:
0 - Magnetom Verio, SIEMENS, 3 T
1 - Magnetom Avanto, SIEMENS, 1,5 Т 
    Siemens MAGNETOM Avanto, 1,5T
2 - Magnetom Symphony, SIEMENS, 1.5 Т
3 - Panorama, PHILIPS, 1.0 Т
4 - None

In [1]:
import numpy as np
import pandas as pd

In [60]:
data = pd.read_csv('/notebook/MS_trainset_anon_csv/reports.csv', sep = '|')
data.head(3)

,id,text,conclusion,label
0,01_414,МАГНИТНО-РЕЗОНАНСНАЯ ТОМОГРАФИЯ ГОЛОВНОГО МОЗ...,ЗАКЛЮЧЕНИЕ: МРТ-данные соответствуют изменени...,ms
1,01_344,МАГНИТНО-РЕЗОНАНСНАЯ ТОМОГРАФИЯ ГОЛОВНОГО МОЗ...,"ЗАКЛЮЧЕНИЕ: МРТ данные, вероятнее всего, соотв...",ms
2,01_316,МАГНИТНО-РЕЗОНАНСНАЯ ТОМОГРАФИЯ ГОЛОВНОГО МОЗГ...,"ЗАКЛЮЧЕНИЕ: МРТ-данные соответствуют, вероятне...",ms


In [39]:
data.label.value_counts()

ms       147
none     130
other     20
Name: label, dtype: int64

In [40]:
data.shape

(297, 4)

In [73]:
data['model'] = data.apply(lambda row: row.text[row.text.find('(')+1 : row.text.find(')')], axis = 1) 

In [74]:
model_code = []
for i in data['model']:
    if i == 'Magnetom Verio, SIEMENS, 3 T' or i == 'Magnetom Verio, SIEMENS, 3 Т':
        model_code.append(0)
    elif i == 'Magnetom Avanto, SIEMENS, 1,5 Т' or i == 'Magnetom Avanto, SIEMENS, 1.5 Т' or i == 'Magnetom Avanto, SIEMENS, 1.5 T' or i == 'Magnetom Avanto, SIEMENS 1,5 T' or i == 'Magnetom Avanto, SIEMENS, 1,5 T' or i == 'Siemens MAGNETOM Avanto, 1,5T':
        model_code.append(1)
    elif i == 'Magnetom Symphony, SIEMENS, 1.5 Т' or i == 'Magnetom Symphony, SIEMENS, 1,5 Т':
        model_code.append(2)
    elif i == 'Panorama, PHILIPS, 1.0 Т':
        model_code.append(3)
    elif i == 'Гадовист 7,5':
        model_code.append(4)

In [75]:
data['model_code'] = model_code

In [76]:
data['model_code'].value_counts()

0    149
1    138
2      7
3      2
4      1
Name: model_code, dtype: int64

# Data preperation

In [52]:
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer('russian')
from nltk import word_tokenize, FreqDist
import string
punc = str.maketrans({key: None for key in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~”„–'})

In [47]:
def stemmer_func(row):
    tmp = []
    for i in row:
        tmp.append(snowball_stemmer.stem(i))
    return tmp

In [48]:
def alphas(row):
    tmp = []
    for i in row:
        if str.isalpha(i):
            tmp.append(i)
    return tmp

In [66]:
#data['splt'] = data.apply(lambda row: (row.text + ' ' + row.conclusion).translate(punc).lower().split(),  axis=1) 
data['splt'] = data.apply(lambda row: row.conclusion.translate(punc).lower().split(),  axis=1) 

In [67]:
data['splt'] = data.apply(lambda row: alphas(row.splt), axis = 1)

In [68]:
data['splt'] = data.apply(lambda row: stemmer_func(row.splt), axis = 1)

In [77]:
data = data[['id', 'label', 'model_code', 'splt']]

In [70]:
tokens = []
for i in range(data.shape[0]):
    for j in data.splt[i]:
        tokens.append(j)

In [71]:
fdist = FreqDist(tokens)

In [72]:
len(fdist)

233

In [87]:
def one_hot_encoder(data, fdist):
    for i in fdist.keys():
        data[i] = data.apply(lambda row: row.splt.count(i), axis = 1)

In [88]:
one_hot_encoder(data, fdist)

In [93]:
data = data.drop(['splt'], axis = 1)
data.head(3)

,id,label,model_code,заключен,мртдан,соответств,изменен,в,обо,полушар,...,месяц,мрпризнак,оториноларинголог,менингиом,случайн,находк,гормональн,статус,кров,лорспециалист
0,01_414,ms,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,01_344,ms,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01_316,ms,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
data.to_csv('/notebook/MS_trainset_anon_csv/one_hot_encoder.csv', sep=',', header=True, index=False, encoding='utf-8')